In [1]:
import os

In [2]:
%pwd

'c:\\Users\\unoto\\Documents\\my_project\\data_science\\Breast-Cancer-Prediction-MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\unoto\\Documents\\my_project\\data_science\\Breast-Cancer-Prediction-MLOps'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    data_clean_path: Path

In [6]:
from breastCancer.constants import *
from breastCancer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            data_clean_path=config.data_clean_path
        )

        return data_transformation_config

In [8]:
import os
from breastCancer import logger
import pandas as pd

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    

    def get_clean_data(self):
        data = pd.read_csv(self.config.data_path)
        data = data.drop(['Unnamed: 32', 'id'], axis=1)

        data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})

        data.to_csv(self.config.data_clean_path, index=False)

        logger.info("Data has been cleaned.")

        return data

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_clean_data()
except Exception as e:
    raise e

[2024-03-03 17:17:47,213: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-03 17:17:47,219: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-03 17:17:47,232: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-03 17:17:47,235: INFO: common: created directory at: artifacts]
[2024-03-03 17:17:47,240: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-03 17:17:47,300: INFO: 1130541284: Data has been cleaned.]
